# Importing Libraries

In [ ]:
import selenium
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchAttributeException
import time
import numpy as np
import pandas as pd
import os

# Preparing chrome webdriver and location for file storing

In [ ]:
# Openning chromedriver & stockbit
driver = webdriver.Chrome()
driver.get('https://stockbit.com/login')
path_file = os.getcwd()
path_file

# Login

In [ ]:
# Login
loginID = '' #fill with your stockbit email
password = '' #fill with your stockbit password
login_box = driver.find_element(By.ID,'username')
login_box.send_keys(loginID)
pass_box = driver.find_element(By.ID,'password')
pass_box.send_keys(password)
pass_box.send_keys(Keys.ENTER)

# Selecting the company and currency

In [ ]:
# Select the company 
comp = 'ASII'
driver.get('https://stockbit.com/#/symbol/'+ comp +'/financials')

In [ ]:
# Select Currency
currency = 'IDR'
se3 = Select(driver.find_element(By.CSS_SELECTOR,"select[data-cy='data-currency']")).select_by_value('IDR')

# Scraping Balance Sheet data

In [ ]:
# Select Report (BS, IS, or CF)

# List of report
# 1 Income Statement
# 2 Balance Sheet
# 3 Cash Flow

se  = Select(driver.find_element(By.CSS_SELECTOR,"select[data-cy='report-type']")).select_by_value('2')

In [ ]:
# Select Period

# There are several choices for period:
# 1 Quarterly
# 2 Annual
# 3 Interim YTD
# 4 Q1
# 5 Q2
# 6 Q3
# 7 Q4
# 8 QoQ Growth
# 9 Quarter YoY Growth
# 10 YTD YoY Growth
# 11 Annual YoY Growth
# 12 3 Year CAGR

period = '2' 
se2 = Select(driver.find_element(By.CSS_SELECTOR,"select[data-cy='statement-type']")).select_by_value(period)

In [ ]:
# Getting header of table
header_1 = driver.find_element(By.CSS_SELECTOR,'span[data-cy="financials-table-title-text"]')
header_2 = driver.find_elements(By.CSS_SELECTOR,'th[data-cy="financials-table-quarter-text"]')
header_text = [header_1.text] + [col.text for col in header_2][16:]

# Click all expand button
for j in range(10):
    i = 0
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR,'i[class="icon-right-dir collapsed"]')
        if len(elements) > i:
            try:
                elements[i].click() # click on the i-th element in the list
                i += 1 # increment i
                time.sleep(2) # wait until list will be updated
            except ElementClickInterceptedException:
                i += 1
            except ElementNotInteractableException:
                i += 1
            continue
        break
    
# Getting table body
data1_bs = []
elements = driver.find_element(By.CSS_SELECTOR,"div[class='tbl-financials    ']").find_elements(By.TAG_NAME,'td')
for element in elements:
    if element.get_attribute('data-value-idr') != None:                                        #Taking numbers data
        if element.get_attribute('data-value-idr') != '-':
            data1_bs.append(element.get_attribute('data-value-idr'))
        else:
            data1_bs.append(None)
    else:                                                                                #Taking account name data
        if element.text != '':
            if 'Total' in element.text and 'Total' not in element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'):
                data1_bs.append('Total ' + element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
            else:
                try:
                    data1_bs.append(element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
                except NoSuchElementException:
                    data1_bs.append(element.text.strip())
        else:
            try:
                data1_bs.append(element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
            except NoSuchElementException:
                data1_bs.append('')

data2_bs = np.array(data1_bs)
data3_bs = data2_bs.reshape(int(len(data1_bs)/len(header_text)),len(header_text))

# Scraping Income Statement data

In [ ]:
# Select Report (BS, IS, or CF)

# List of report
# 1 Income Statement
# 2 Balance Sheet
# 3 Cash Flow

se  = Select(driver.find_element(By.CSS_SELECTOR,"select[data-cy='report-type']")).select_by_value('1')

In [ ]:
# Getting header of table
header_1 = driver.find_element(By.CSS_SELECTOR,'span[data-cy="financials-table-title-text"]')
header_2 = driver.find_elements(By.CSS_SELECTOR,'th[data-cy="financials-table-quarter-text"]')
header_text = [header_1.text] + [col.text for col in header_2][16:]

# Click all expand button
for j in range(10):
    i = 0
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR,'i[class="icon-right-dir collapsed"]')
        if len(elements) > i:
            try:
                elements[i].click() # click on the i-th element in the list
                i += 1 # increment i
                time.sleep(2) # wait until list will be updated
            except ElementClickInterceptedException:
                i += 1
            except ElementNotInteractableException:
                i += 1
            continue
        break
    
# Getting table body
data1_is = []
elements = driver.find_element(By.CSS_SELECTOR,"div[class='tbl-financials    ']").find_elements(By.TAG_NAME,'td')
for element in elements:
    if element.get_attribute('data-value-idr') != None:                                        #Taking numbers data
        if element.get_attribute('data-value-idr') != '-':
            data1_is.append(element.get_attribute('data-value-idr'))
        else:
            data1_is.append(None)
    else:                                                                                #Taking account name data
        if element.text != '':
            if 'Total' in element.text and 'Total' not in element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'):
                data1_is.append('Total ' + element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
            else:
                try:
                    data1_is.append(element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
                except NoSuchElementException:
                    data1_is.append(element.text.strip())
        else:
            try:
                data1_is.append(element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
            except NoSuchElementException:
                data1_is.append('')

data2_is = np.array(data1_is)
data3_is = data2_is.reshape(int(len(data1_is)/len(header_text)),len(header_text))

# Scraping Cash Flow data

In [ ]:
# Select Report (BS, IS, or CF)

# List of report
# 1 Income Statement
# 2 Balance Sheet
# 3 Cash Flow
se  = Select(driver.find_element(By.CSS_SELECTOR,"select[data-cy='report-type']")).select_by_value('3')

In [ ]:
# Getting header of table
header_1 = driver.find_element(By.CSS_SELECTOR,'span[data-cy="financials-table-title-text"]')
header_2 = driver.find_elements(By.CSS_SELECTOR,'th[data-cy="financials-table-quarter-text"]')
header_text = [header_1.text] + [col.text for col in header_2][16:]

# Click all expand button
for j in range(10):
    i = 0
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR,'i[class="icon-right-dir collapsed"]')
        if len(elements) > i:
            try:
                elements[i].click() # click on the i-th element in the list
                i += 1 # increment i
                time.sleep(2) # wait until list will be updated
            except ElementClickInterceptedException:
                i += 1
            except ElementNotInteractableException:
                i += 1
            continue
        break
    
# Getting table body
data1_cf = []
elements = driver.find_element(By.CSS_SELECTOR,"div[class='tbl-financials  disabled-percentage  ']").find_elements(By.TAG_NAME,'td')
for element in elements:
    if element.get_attribute('data-value-idr') != None:                                        #Taking numbers data
        if element.get_attribute('data-value-idr') != '-':
            data1_cf.append(element.get_attribute('data-value-idr'))
        else:
            data1_cf.append(None)
    else:                                                                                #Taking account name data
        if element.text != '':
            if 'Total' in element.text and 'Total' not in element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'):
                data1_cf.append('Total ' + element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
            else:
                try:
                    data1_cf.append(element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
                except NoSuchElementException:
                    data1_cf.append(element.text.strip())
        else:
            try:
                data1_cf.append(element.find_element(By.CLASS_NAME,'acc-name').get_attribute('title'))
            except NoSuchElementException:
                data1_cf.append('')

data2_cf = np.array(data1_cf)
data3_cf = data2_cf.reshape(int(len(data1_cf)/len(header_text)),len(header_text))

In [ ]:
# Store balance sheet, income statement, cash flow into dataframe
bs = pd.DataFrame(data3_bs,columns=header_text)
ist = pd.DataFrame(data3_is,columns=header_text)
cf = pd.DataFrame(data3_cf,columns=header_text)

In [ ]:
# Function to change number from strings format to integer
def change_to_num_bill(x):
    if x not in ('',None,np.nan):
        try:
            return int(x)/1000000000
        except ValueError:
            return x/1000000000
    else :
        return np.nan

In [ ]:
# Apply change format function to the dataframe
for col in header_text[1:]:
    bs[col] = bs[col].apply(lambda x: change_to_num_bill(x)).round(0)
    ist[col] = ist[col].apply(lambda x: change_to_num_bill(x)).round(0)
    cf[col] = cf[col].apply(lambda x: change_to_num_bill(x)).round(0)

In [ ]:
with pd.ExcelWriter(path_file +'\\'+ comp + '.xlsx') as writer:
    bs.to_excel(writer, sheet_name='Balance Sheet',index=False)
    ist.to_excel(writer, sheet_name='Income Statement',index=False)
    cf.to_excel(writer, sheet_name='Cash Flow',index=False)